## SPARK SQL

# WHAT CUSTOMERS MOSTLY BUY TOGETHER

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("Customer_trolley").getOrCreate()
dataset = spark.read.csv(r"C:\Users\stytom\Plocha\bought_togother.csv", inferSchema=True, header=True)

In [3]:
# raw_data
dataset.createOrReplaceTempView("data")

spark.sql("""
SELECT *
FROM data

""").show()

+--------+---+
|order_id|sku|
+--------+---+
|       1|  A|
|       1|  B|
|       1|  C|
|       2|  B|
|       2|  A|
|       3|  A|
|       4|  B|
|       4|  C|
+--------+---+



In [4]:
dataset.createOrReplaceTempView("items")

spark.sql("""
SELECT a.sku orig_sku, b.sku bought_with, count(*) times_togother
FROM items as a
inner join items as b on a.order_id = b.order_id and a.sku != b.sku
group by a.sku, b.sku

""").show()

+--------+-----------+--------------+
|orig_sku|bought_with|times_togother|
+--------+-----------+--------------+
|       B|          C|             2|
|       A|          C|             1|
|       C|          B|             2|
|       C|          A|             1|
|       B|          A|             2|
|       A|          B|             2|
+--------+-----------+--------------+



## CLIENTS BETTING AGAINS THEMSELVES

In [5]:
dataset1 = spark.read.csv(r"C:\Users\stytom\Plocha\matches.csv", inferSchema=True, header=True)

In [6]:
dataset1.createOrReplaceTempView("data")

spark.sql("""
SELECT *
FROM data

""").show()

+--------------------+------+--------+--------------+--------------------+---+-----+
|number of the ticket|client|   sport|        league|               teams|tip|stake|
+--------------------+------+--------+--------------+--------------------+---+-----+
|                 222|  1111|football|Premier league|    Brighton - Leeds|  1|  165|
|                 223|  2222|football|Premier league|    Brighton - Leeds|  1|  440|
|                 224|  3333|football|Premier league|    Brighton - Leeds|  1|  320|
|                 225|  4444|football|Premier league|    Brighton - Leeds|  2|  622|
|                 226|  1111|  hockey|           NHL|    Ducks - Penguins|  1|  206|
|                 227|  2222|  hockey|           NHL|    Ducks - Penguins|  1|  257|
|                 228|  3333|  hockey|           NHL|   Senators - Flyers|  2|  937|
|                 229|  4444|  hockey|           NHL|   Senators - Flyers|  1|  590|
|                 230|  1111|handball|       Germany|    Munchen 

In [7]:
dataset1.createOrReplaceTempView("data")

spark.sql("""


SELECT *
FROM (SELECT client, league, teams, tip FROM data) pc1
JOIN (SELECT client, league, teams, tip FROM data) pc2 on pc1.client > pc2.client



""").show()

+------+--------------+----------------+---+------+--------------+--------------------+---+
|client|        league|           teams|tip|client|        league|               teams|tip|
+------+--------------+----------------+---+------+--------------+--------------------+---+
|  2222|Premier league|Brighton - Leeds|  1|  1111|Premier league|    Brighton - Leeds|  1|
|  2222|Premier league|Brighton - Leeds|  1|  1111|           NHL|    Ducks - Penguins|  1|
|  2222|Premier league|Brighton - Leeds|  1|  1111|       Germany|    Munchen - Berlin|  1|
|  2222|Premier league|Brighton - Leeds|  1|  1111|Premier league|   Chelsea - Norwich|  0|
|  2222|Premier league|Brighton - Leeds|  1|  1111|    Bundesliga|Dortmund - Norimberk|  2|
|  2222|Premier league|Brighton - Leeds|  1|  1111|    Bundesliga|Wolfsburg - Frank...|  1|
|  3333|Premier league|Brighton - Leeds|  1|  1111|Premier league|    Brighton - Leeds|  1|
|  3333|Premier league|Brighton - Leeds|  1|  2222|Premier league|    Brighton -

In [8]:
dataset1.createOrReplaceTempView("data")

spark.sql("""

SELECT pc1.client client_1, pc2.client client_2, COUNT(DISTINCT pc1.league, pc1.teams) pc1_tickets, 
    COUNT(DISTINCT pc2.league, pc2.teams) pc2_tickets, 
    SUM(CASE WHEN pc1.teams = pc2.teams then 1 else 0 end) same_market,
    
    SUM(CASE WHEN pc1.teams = pc2.teams and pc1.tip = pc2.tip then 1 else 0 end) same_tip,
    SUM(CASE WHEN pc1.teams = pc2.teams and pc1.tip != pc2.tip then 1 else 0 end) diff_tip
    
FROM (SELECT client, league, teams, tip FROM data) pc1
JOIN (SELECT client, league, teams, tip FROM data) pc2 on pc1.client > pc2.client
GROUP BY pc1.client, pc2.client
ORDER BY 6 desc

""").show()

+--------+--------+-----------+-----------+-----------+--------+--------+
|client_1|client_2|pc1_tickets|pc2_tickets|same_market|same_tip|diff_tip|
+--------+--------+-----------+-----------+-----------+--------+--------+
|    3333|    2222|          8|          9|          4|       2|       2|
|    3333|    1111|          8|          6|          2|       2|       0|
|    2222|    1111|          9|          6|          5|       2|       3|
|    4444|    2222|          5|          9|          4|       1|       3|
|    4444|    3333|          5|          8|          5|       0|       5|
|    4444|    1111|          5|          6|          3|       0|       3|
+--------+--------+-----------+-----------+-----------+--------+--------+

